In [1]:
!pip uninstall numpy pybind11 gensim contourpy numba datasets -y
!pip install numpy pybind11 gensim contourpy numba datasets evaluate transformers
!pip install "accelerate>=0.26.0"
!pip install transformers[torch]

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pybind11 2.13.6
Uninstalling pybind11-2.13.6:
  Successfully uninstalled pybind11-2.13.6
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: contourpy 1.3.1
Uninstalling contourpy-1.3.1:
  Successfully uninstalled contourpy-1.3.1
Found existing installation: numba 0.61.0
Uninstalling numba-0.61.0:
  Successfully uninstalled numba-0.61.0
Found existing installation: datasets 3.2.0
Uninstalling datasets-3.2.0:
  Successfully uninstalled datasets-3.2.0
  Using cached numpy-2.2.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached gensim-4.3.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.1 kB)
  Using cached contourpy-1.3.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached 

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
import numpy as np
from evaluate import load
from transformers import TrainingArguments, Trainer
from evaluate import load

In [3]:
tomatoes = load_dataset("rotten_tomatoes")
train_data, test_data = tomatoes["train"], tomatoes["test"]

In [4]:
model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(
                                                model_id, num_labels=2
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
def preprocess_function(examples):
    """Tokenize input data"""
    return tokenizer(examples["text"], truncation=True)

In [7]:
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [8]:
def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    load_f1 = load("f1")
    f1 = load_f1.compute(predictions=predictions,
    references=labels)["f1"]
    return {"f1": f1}

In [9]:
training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/var/folders/_b/drkg0vjx1v368n_9th3dhc7c0000gn/T/ipykernel_60564/4214145445.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

Step,Training Loss
500,0.416400


TrainOutput(global_step=534, training_loss=0.4124848958704802, metrics={'train_runtime': 113.9643, 'train_samples_per_second': 74.848, 'train_steps_per_second': 4.686, 'total_flos': 227605451772240.0, 'train_loss': 0.4124848958704802, 'epoch': 1.0})

In [12]:
trainer.evaluate()

{'eval_loss': 0.36203113198280334,
 'eval_f1': 0.8494423791821561,
 'eval_runtime': 5.0899,
 'eval_samples_per_second': 209.433,
 'eval_steps_per_second': 13.163,
 'epoch': 1.0}

##### Freezing layers

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
for name, param in model.named_parameters():
    print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [21]:
for name, param in model.named_parameters():
    # Trainable classification head
    if name.startswith("classifier"):
        param.requires_grad = True
    # Freeze everything else
    else:
        param.requires_grad = False

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/var/folders/_b/drkg0vjx1v368n_9th3dhc7c0000gn/T/ipykernel_60564/3933461971.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.695000


TrainOutput(global_step=534, training_loss=0.6943904076622667, metrics={'train_runtime': 32.465, 'train_samples_per_second': 262.744, 'train_steps_per_second': 16.448, 'total_flos': 227605451772240.0, 'train_loss': 0.6943904076622667, 'epoch': 1.0})

In [23]:
trainer.evaluate()

{'eval_loss': 0.6818975806236267,
 'eval_f1': 0.6308724832214765,
 'eval_runtime': 3.9831,
 'eval_samples_per_second': 267.63,
 'eval_steps_per_second': 16.821,
 'epoch': 1.0}

#### Few shot classification (SetFit)

In [40]:
from setfit import sample_dataset
from setfit import SetFitModel
from setfit import TrainingArguments as SetFitTrainingArguments
from setfit import Trainer as SetFitTrainer

In [41]:
sampled_train_data = sample_dataset(tomatoes["train"], num_samples=16)

In [42]:
# Load a pretrained SentenceTransformer model
model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [43]:
args = SetFitTrainingArguments(
    num_epochs=3, # The number of epochs to use for contrastive learning
    num_iterations=20 # The number of text pairs to generate
)
args.eval_strategy = args.evaluation_strategy

In [44]:
# Create trainer
trainer = SetFitTrainer(
    model=model,
    args=args,
    train_dataset=sampled_train_data,
    eval_dataset=test_data,
    metric="f1"
)

/opt/anaconda3/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [45]:
trainer.train()

***** Running training *****
  Num unique pairs = 1280
  Batch size = 16
  Num epochs = 3


Step,Training Loss,Validation Loss


In [46]:
trainer.evaluate()

***** Running evaluation *****


{'f1': 0.8519230769230769}

#### Continued Pretraining with Masked Language Modeling

In [68]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForSequenceClassification

In [69]:
# Load model for masked language modeling (MLM)
model = AutoModelForMaskedLM.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [70]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [71]:
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_train = tokenized_train.remove_columns("label")
tokenized_test = test_data.map(preprocess_function, batched=True)
tokenized_test = tokenized_test.remove_columns("label")

/opt/anaconda3/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


In [72]:
# Masking Tokens
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [73]:
training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

In [74]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/var/folders/_b/drkg0vjx1v368n_9th3dhc7c0000gn/T/ipykernel_60564/44570168.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [75]:
# Save pre-trained tokenizer
tokenizer.save_pretrained("mlm")

('mlm/tokenizer_config.json',
 'mlm/special_tokens_map.json',
 'mlm/vocab.txt',
 'mlm/added_tokens.json',
 'mlm/tokenizer.json')

In [76]:
# Train model
trainer.train()

Step,Training Loss
500,2.603300
1000,2.377200
1500,2.299400
2000,2.188800
2500,2.139100
3000,2.094800
3500,2.058500
4000,1.988300
4500,1.978500
5000,1.966700


TrainOutput(global_step=5340, training_loss=2.1563735561870905, metrics={'train_runtime': 1644.1685, 'train_samples_per_second': 51.88, 'train_steps_per_second': 3.248, 'total_flos': 2273819807394096.0, 'train_loss': 2.1563735561870905, 'epoch': 10.0})

In [77]:
model.save_pretrained("mlm")

In [78]:
model = AutoModelForSequenceClassification.from_pretrained("mlm", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("mlm")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mlm and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [95]:
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

/opt/anaconda3/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [96]:
training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

In [97]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/var/folders/_b/drkg0vjx1v368n_9th3dhc7c0000gn/T/ipykernel_60564/4214145445.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [98]:
trainer.train()

Step,Training Loss
500,0.371600


TrainOutput(global_step=534, training_loss=0.36948965551254903, metrics={'train_runtime': 115.8636, 'train_samples_per_second': 73.621, 'train_steps_per_second': 4.609, 'total_flos': 243073504050240.0, 'train_loss': 0.36948965551254903, 'epoch': 1.0})

In [99]:
trainer.evaluate()

{'eval_loss': 0.36133506894111633,
 'eval_f1': 0.8536585365853658,
 'eval_runtime': 5.1914,
 'eval_samples_per_second': 205.339,
 'eval_steps_per_second': 12.906,
 'epoch': 1.0}